In [246]:
from pydantic import BaseModel,Field
from typing import List, Optional,Dict
from enum import Enum

class ExpertiseLevel(str, Enum):
    BEGINNER = "beginner"
    INTERMEDIATE = "intermediate"
    ADVANCED = "advanced"

# ====== Fixed Learning Material Model ======
class LearningMaterial(BaseModel):
    title: str
    url: str
    type: str=Field(...,description="video, article, or exercise")
    description: str

class MaterialCollection(BaseModel):  # Renamed for clarity
    materials: List[LearningMaterial]

class QuizQuestion(BaseModel):
    question: str
    options: List[str]
    correct_answer: int
    explanation: str

class QuizFormat(BaseModel):
    questions: List[QuizQuestion]


class Quiz(BaseModel):
    topic: str
    questions: List[QuizQuestion]
    
# Fix Quiz model structure
class QuizQuestion(BaseModel):
    question: str
    options: List[str]
    correct_answer: int  # Index of correct option (0-based)
    explanation: str

class Quiz(BaseModel):  # Renamed from QuizFormat
    questions: List[QuizQuestion]

# Corrected ProjectIdea model (represents a SINGLE project)
class ProjectIdea(BaseModel):
    title: str
    description: str
    difficulty: ExpertiseLevel
    estimated_duration: str = Field(..., description="Duration estimation in days")
    required_skills: List[str]
    learning_outcomes: List[str]

# Container for multiple projects
class Projects(BaseModel):
    projects: List[ProjectIdea]

In [247]:
from crewai import Agent,Crew,Task,LLM,Process
from typing import List
import os
from crewai_tools import SerperDevTool
#
from dotenv import load_dotenv
load_dotenv()
llm = LLM('ollama/llama3.2',
          temperature=0.7)

In [248]:
search_tool = SerperDevTool(serper_api_key=os.getenv("SERPER_API_KEY"))

In [309]:
learning_material_agent =Agent(
            role='Learning Material Curator',
            goal='Curate high-quality learning materials based on user topics and expertise level',
            backstory="""You are an expert educational content curator with years of experience
            in finding the best learning resources for students at different levels. You know how
            to identify reliable and high-quality educational content from reputable sources.""",
            llm=llm,
            verbose=True
        )

quiz_creator_agent= Agent(
            role='Quiz Creator',
            goal='Create engaging and educational quizzes to test understanding',
            backstory="""You are an experienced educator who specializes in creating
            effective assessment questions that test understanding while promoting learning.""",
            llm=llm,
            verbose=True
        )

project_suggestion_agent=  Agent(
            role='Project Advisor',
            goal='Suggest practical projects that match user expertise and interests',
            backstory="""You are a project-based learning expert who knows how to create
            engaging hands-on projects that reinforce learning objectives.""",
            llm=llm,
            verbose=True
        ) 

In [310]:
print(search_tool.run(search_query="Data Structures using Python for intermediate level",num=5))

Using Tool: Search the internet with Serper
{'searchParameters': {'q': 'Data Structures using Python for intermediate level', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Python Data Structures | GeeksforGeeks', 'link': 'https://www.geeksforgeeks.org/python-data-structures/', 'snippet': 'We will discuss all the in-built data structures like list tuples, dictionaries, etc. as well as some advanced data structures like trees, graphs, etc.', 'position': 1, 'sitelinks': [{'title': 'Inbuilt Data Structures in...', 'link': 'https://www.geeksforgeeks.org/inbuilt-data-structures-python/'}, {'title': 'User Defined Data Structures...', 'link': 'https://www.geeksforgeeks.org/user-defined-data-structures-in-python/'}]}, {'title': '2025 Data Structures Using Python | Udemy', 'link': 'https://www.udemy.com/course/data-structures-using-python/?srsltid=AfmBOopwmn6Jj1z16be16hQTWe2Y_eMTKxfXAId8cX4a8DQWwSuHosJr', 'snippet': "From arrays and linked lists to stacks, queues, tree

In [311]:
from crewai.utilities import Converter

class ValidatedConverter(Converter):
    def to_pydantic(self, current_attempt=1):
        try:
            # Clean and validate JSON structure
            cleaned = self.content.replace("'", '"').replace("True", "true").replace("False", "false")
            return super().to_pydantic(current_attempt)
        except Exception as e:
            print(f"Validation failed: {str(e)}")
            raise

In [315]:
create_learning_material_task =   Task(
            description=""""{topics}.
            Explain {topics}  to a {expertise_level} level
            Include a mix of videos, articles, and practical exercises.
            Ensure all materials are from reputable sources and are current.
            Include GitHub repos for practical exercises.Verify source credibility before including.
             Format response as: {{
            "materials": [
                {{
                    "title": "...",
                    "url": "...",
                    "type": "...",
                    "description": "..."
                }}
            ]
        }}""",
           expected_output="""
                    {
                        "materials": [
                            {
                                "title": "Titel1",
                                "url": "url1",
                                "type": "article/video",
                                "description": "In-depth description of the content with examples"
                            },
                            {
                                "title": "title2n",
                                "url": "url2",
                                "type": "article/video",
                                "description": "In-depth description of the content with examples"
                            }
                        ]
                    }
            """,
            agent=learning_material_agent,
            tools=[search_tool],
        )

create_quiz_task = Task(
    description="""Create a comprehensive quiz for the topics: {topics}.""",
    expected_output="""
    {
        "questions": [
            {
                "question": "What is a Python list?",
                "options": ["Immutable collection", "Mutable collection", "Key-value pairs"],
                "correct_answer": 1,
                "explanation": "Python lists are mutable ordered collections"
            }
        ]
    }
    """,
    agent=quiz_creator_agent,
    output_pydantic=QuizFormat  # Corrected model name
)

# Update quiz task
create_quiz_task = Task(
    description="""Create a quiz for {topics} at {expertise_level} level.""",
    expected_output="""
    {
        "questions": [
            {
                "question": "Question1",
                "options": [OPTION1 ,OPTION2,OPTION3,OPTION4,OPTION5],
                "correct_answer": OPTION,
                "explanation": "Explanation of the correct answer option"
            },
            stions": [
            {
                "question": "Question1",
                "options": [OPTION1 ,OPTION2,OPTION3,OPTION4,OPTION5],
                "correct_answer": OPTION,
                "explanation": "Explanation of the correct answer option"
            },
            ...
        ]
    }
    """,
    agent=quiz_creator_agent,
    output_pydantic=Quiz  # Match model name
)
create_project_suggestion_task = Task(
    description="""Suggest ONLY 5 BEST practical project ideas for the topics: {topics}.
    Projects should be suitable for {expertise_level} level.
    Include title,description,difficulty,estimated duration, required skills, and learning outcomes.
    Suggest projects that have recent community activity (check GitHub).Include links to relevant documentation.
    Projects should be engaging and reinforce key concepts.""",
    expected_output="""
    {
        "projects": [
            {
                "title": "Project 1 Title",
                "description": "Description 1",
                "difficulty": "intermediate",
                "estimated_duration": "2 weeks",
                "required_skills": ["Python", "Data Structures"],
                "learning_outcomes": ["Understand hash tables", "Implement trees in Python"]
            },
            {
                "title": "Project 2 Title",
                "description": "Description 2",
                "difficulty": "intermediate",
                "estimated_duration": "3 weeks",
                "required_skills": ["OOP", "Algorithms"],
                "learning_outcomes": ["Develop sorting algorithms", "Optimize code efficiency"]
            }
        ]
    }
    """,
    agent=project_suggestion_agent,
    output_pydantic=Projects  # Changed to Projects model
)

In [316]:

        
# Create and run the crew
crew = Crew(
agents=[learning_material_agent, quiz_creator_agent, project_suggestion_agent],
tasks=[create_learning_material_task,create_quiz_task,create_project_suggestion_task],
process=Process.sequential,
)
        
       

In [328]:
inputs = {"topics":"Openai whisper",
          "expertise_level":"INTERMEDIATE"} 
result = crew.kickoff(inputs=inputs)


# Agent: Learning Material Curator
## Task: "Openai whisper.
            Explain Openai whisper  to a INTERMEDIATE level
            Include a mix of videos, articles, and practical exercises.
            Ensure all materials are from reputable sources and are current.
            Include GitHub repos for practical exercises.Verify source credibility before including.
             Format response as: {{
            "materials": [
                {{
                    "title": "...",
                    "url": "...",
                    "type": "...",
                    "description": "..."
                }}
            ]
        }}


# Agent: Learning Material Curator
## Thought: Action: Search the internet with Serper
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Openai whisper intermediate explanation\"}"
## Tool Output: 
{'searchParameters': {'q': 'Openai whisper intermediate explanation', 'type': 'search', 'num': 10, 'engine': 'google'}, 'o

In [329]:
print(result.raw)

{
  "projects": [
    {
      "title": "Audio Transcription using OpenAI Whisper",
      "description": "In this project, you will be working with OpenAI Whisper to transcribe audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate the transcription. You can use this project to practice your skills in preprocessing data, splitting audio into chunks, and using the Whisper model for transcription.",
      "difficulty": "intermediate",
      "estimated_duration": "2 weeks",
      "required_skills": ["Python", "Data Structures"],
      "learning_outcomes": ["Understand hash tables", "Implement trees in Python"]
    },
    {
      "title": "Sentiment Analysis using OpenAI Whisper",
      "description": "In this project, you will be working with OpenAI Whisper to perform sentiment analysis on audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to

In [330]:
print(result.pydantic.projects)

[ProjectIdea(title='Audio Transcription using OpenAI Whisper', description='In this project, you will be working with OpenAI Whisper to transcribe audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate the transcription. You can use this project to practice your skills in preprocessing data, splitting audio into chunks, and using the Whisper model for transcription.', difficulty=<ExpertiseLevel.INTERMEDIATE: 'intermediate'>, estimated_duration='2 weeks', required_skills=['Python', 'Data Structures'], learning_outcomes=['Understand hash tables', 'Implement trees in Python']), ProjectIdea(title='Sentiment Analysis using OpenAI Whisper', description='In this project, you will be working with OpenAI Whisper to perform sentiment analysis on audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate a transcription. You can then use natural

In [331]:
print(len(result.pydantic.projects))

5


In [332]:
print("="*80)
for items in result.pydantic.projects:
    print(items.title)
    print(items.description)
    print(items.difficulty)
    print(items.estimated_duration)
    print(items.required_skills)
    print(items.learning_outcomes)
    print("="*80)

Audio Transcription using OpenAI Whisper
In this project, you will be working with OpenAI Whisper to transcribe audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate the transcription. You can use this project to practice your skills in preprocessing data, splitting audio into chunks, and using the Whisper model for transcription.
ExpertiseLevel.INTERMEDIATE
2 weeks
['Python', 'Data Structures']
['Understand hash tables', 'Implement trees in Python']
Sentiment Analysis using OpenAI Whisper
In this project, you will be working with OpenAI Whisper to perform sentiment analysis on audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate a transcription. You can then use natural language processing techniques to analyze the sentiment of the transcript.
ExpertiseLevel.INTERMEDIATE
3 weeks
['OOP', 'Algorithms']
['Develop sorting algorith

In [333]:
print(create_learning_material_task.description)

"Openai whisper.
            Explain Openai whisper  to a INTERMEDIATE level
            Include a mix of videos, articles, and practical exercises.
            Ensure all materials are from reputable sources and are current.
            Include GitHub repos for practical exercises.Verify source credibility before including.
             Format response as: {{
            "materials": [
                {{
                    "title": "...",
                    "url": "...",
                    "type": "...",
                    "description": "..."
                }}
            ]
        }}


In [334]:
print(create_learning_material_task.output.raw)

Based on the search results, here's a summary of what I found:

**What is OpenAI Whisper?**

OpenAI Whisper is an advanced automatic speech recognition (ASR) model that can transcribe audio into text in multiple languages and even translate it into English.

**How does Whisper work?**

Whisper uses a simple end-to-end approach implemented as an encoder-decoder Transformer. It splits input audio into 30-second chunks, converts them into log-Mel spectrograms, and then passes them through an encoder to generate the final transcription.

**Advantages of Whisper over Google Speech-to-Text:**

Whisper prioritizes privacy, offline use, and cost-efficiency, making it a better choice for scenarios where these factors are important. However, it may not be as scalable or cloud-integrated as Google Speech-to-Text for real-time applications.

**What is the purpose of Whisper?**

Whisper is not a social app, but rather an AI-powered tool designed to understand and transcribe spoken language. It can 

In [335]:
print(create_quiz_task.output.raw)

{
  "questions": [
    {
      "question": "What is OpenAI Whisper?",
      "options": ["An advanced speech recognition model", "A simple end-to-end approach implemented as an encoder-decoder Transformer", "A tool for social media", "An AI-powered audio transcription software", "A language translation app"],
      "correct_answer": 0,
      "explanation": "OpenAI Whisper is an advanced automatic speech recognition (ASR) model that can transcribe audio into text in multiple languages and even translate it into English."
    },
    {
      "question": "How does Whisper work?",
      "options": ["It splits input audio into 30-second chunks, converts them into log-Mel spectrograms, and then passes them through an encoder to generate the final transcription.", "It uses a complex neural network to learn patterns in speech.", "It prioritizes scalability over accuracy.", "It is designed for real-time applications only.", "It uses a simple approach implemented as an encoder-decoder Transformer.

In [336]:
print(create_project_suggestion_task.output.raw)

{
  "projects": [
    {
      "title": "Audio Transcription using OpenAI Whisper",
      "description": "In this project, you will be working with OpenAI Whisper to transcribe audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to generate the transcription. You can use this project to practice your skills in preprocessing data, splitting audio into chunks, and using the Whisper model for transcription.",
      "difficulty": "intermediate",
      "estimated_duration": "2 weeks",
      "required_skills": ["Python", "Data Structures"],
      "learning_outcomes": ["Understand hash tables", "Implement trees in Python"]
    },
    {
      "title": "Sentiment Analysis using OpenAI Whisper",
      "description": "In this project, you will be working with OpenAI Whisper to perform sentiment analysis on audio files. You will need to preprocess the audio data, split it into chunks, and then pass it through the Whisper model to

In [340]:
print("="*80)
for items in create_quiz_task.output.pydantic.questions:
    print(items.question)
    print(items.options)
    print(items.correct_answer)
    print(items.explanation)
    print("="*80)

What is OpenAI Whisper?
['An advanced speech recognition model', 'A simple end-to-end approach implemented as an encoder-decoder Transformer', 'A tool for social media', 'An AI-powered audio transcription software', 'A language translation app']
0
OpenAI Whisper is an advanced automatic speech recognition (ASR) model that can transcribe audio into text in multiple languages and even translate it into English.
How does Whisper work?
['It splits input audio into 30-second chunks, converts them into log-Mel spectrograms, and then passes them through an encoder to generate the final transcription.', 'It uses a complex neural network to learn patterns in speech.', 'It prioritizes scalability over accuracy.', 'It is designed for real-time applications only.', 'It uses a simple approach implemented as an encoder-decoder Transformer.']
0
Whisper uses a simple end-to-end approach implemented as an encoder-decoder Transformer. It splits input audio into 30-second chunks, converts them into log-M